<a href="https://colab.research.google.com/github/Maman08/dailyCode/blob/main/optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p Data



In [2]:
!mkdir -p checkpoints


In [3]:
!wget -O Data/winequality-red.csv https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

--2025-05-17 09:01:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘Data/winequality-red.csv’

Data/winequality-re     [ <=>                ]  82.23K   465KB/s    in 0.2s    

2025-05-17 09:01:03 (465 KB/s) - ‘Data/winequality-red.csv’ saved [84199]



In [27]:
from re import VERBOSE
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping , ModelCheckpoint
from keras.optimizers import  SGD , Adam , RMSprop , Nadam , Adamax
SEED =2017
np.random.seed(SEED)
data = pd.read_csv('/content/Data/winequality-red.csv' ,sep=';')
y= data['quality']
x= data.drop(['quality'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=SEED)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=SEED)
def create_model(optimizer_name):
  model = Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dense(1, activation='linear')
])

  return model
optimizers_Aval = {
    'sgd': SGD(learning_rate=0.001),
    'adam' : Adam(),
    'adamx' : Adamax(),
    'rmsprop' : RMSprop(),
    'nadam' : Nadam()
}
results= []
for opt in optimizers_Aval:
  print(f"Training with optimizer : {opt}")
  model = create_model(opt)
  model.compile(loss='mse' , optimizer=optimizers_Aval[opt],metrics=['mae'])
  early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True)

  hist = model.fit(x_train.values , y_train , epochs=5 ,callbacks=[early_stopping] ,validation_data= (x_val.values,y_val))
  # Get the best epoch and validation loss
  best_epoch = np.argmin(hist.history['val_loss'])
  best_val_loss = np.min(hist.history['val_loss'])

  # Evaluate on test set
  test_loss, test_acc = model.evaluate(x_test.values, y_test, verbose=0)

  # Append results
  results.append({
      'optimizer': opt,
      'best_epoch': best_epoch,
      'best_val_loss': best_val_loss,
      'test_loss': test_loss,
      'test_accuracy': test_acc
  })

# Convert to DataFrame for easier viewing
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='test_loss'))


Training with optimizer : sgd
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 23.1212 - mae: 4.5521 - val_loss: 14.5424 - val_mae: 3.2519
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.0627 - mae: 2.0949 - val_loss: 9.2689 - val_mae: 2.6139
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.6395 - mae: 1.5932 - val_loss: 5.2325 - val_mae: 1.9352
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6019 - mae: 1.2869 - val_loss: 3.5678 - val_mae: 1.5883
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0859 - mae: 1.1674 - val_loss: 2.3592 - val_mae: 1.2593
Training with optimizer : adam
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 15.2556 - mae: 3.5422 - val_loss: 26.4438 - val_mae: 4.2149
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 4.0819 - mae: 1.6315 - val_loss: 7.2731 - val_mae: 2.3220
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1203 - mae: 1.1597 - val_loss: 3.0837 - val_mae: 1.4421
Epoch 4/5
32/